In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from numpy import math
from datetime import datetime, date, time
from datetime import timedelta

from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics

In [2]:
header_list = ['Date Time', 'Voltage', 'Current']
df = pd.read_csv('sensors_data.csv', names = header_list)
df['Power (W)'] = df['Voltage'] * df['Current']
df['Power (KW)'] = df['Power (W)'] / 1000
df['Date Time'] = pd.to_datetime(df['Date Time'])
df['Date'] = df['Date Time'].dt.date
df['Date'] = pd.to_datetime(df['Date'])
df['Time'] = pd.to_datetime(df['Date Time']).dt.time
df['Hour'] = pd.to_datetime(df['Date Time']).dt.hour
df['Time'] = df['Time'].astype(str)
# df['Hour'] = df['Hour'].astype(str)

In [3]:
df.head(5)

,Date Time,Voltage,Current,Power (W),Power (KW),Date,Time,Hour
0,2022-06-24 23:31:26,0.26855,0.0,0.0,0.0,2022-06-24,23:31:26,23
1,2022-06-25 00:31:26,0.26855,0.0,0.0,0.0,2022-06-25,00:31:26,0
2,2022-06-25 01:31:26,0.26855,0.0,0.0,0.0,2022-06-25,01:31:26,1
3,2022-06-25 01:32:26,0.26855,0.0,0.0,0.0,2022-06-25,01:32:26,1
4,2022-06-25 01:33:26,0.31738,0.0,0.0,0.0,2022-06-25,01:33:26,1


In [4]:
rearrange_columns = ['Date Time', 'Date', 'Time', 'Hour', 'Voltage', 'Current', 'Power (W)', 'Power (KW)']
df = df[rearrange_columns]
df.tail(5)

,Date Time,Date,Time,Hour,Voltage,Current,Power (W),Power (KW)
7551,2022-06-30 16:01:22,2022-06-30,16:01:22,16,13.20801,0.17780,2.348384,0.002348
7552,2022-06-30 16:02:22,2022-06-30,16:02:22,16,13.20801,0.18669,2.465803,0.002466
7553,2022-06-30 16:03:22,2022-06-30,16:03:22,16,13.23242,0.19150,2.534008,0.002534
7554,2022-06-30 16:04:22,2022-06-30,16:04:22,16,13.23242,0.19728,2.610492,0.002610
7555,2022-06-30 16:05:23,2022-06-30,16:05:23,16,13.20801,0.19632,2.592997,0.002593


In [5]:
unique_date = df['Date'].unique()
unique_date

array(['2022-06-24T00:00:00.000000000', '2022-06-25T00:00:00.000000000',
       '2022-06-26T00:00:00.000000000', '2022-06-27T00:00:00.000000000',
       '2022-06-28T00:00:00.000000000', '2022-06-29T00:00:00.000000000',
       '2022-06-30T00:00:00.000000000'], dtype='datetime64[ns]')

In [6]:
filter_today_values = df[df['Date'] == unique_date[-1]][['Date', 'Hour', 'Power (KW)']]
today_hourly_values = filter_today_values.groupby(['Date', 'Hour'])['Power (KW)'].sum().reset_index()
length_today_hourly_values = len(today_hourly_values)
length_today_hourly_values
# today_hourly_values

17

In [7]:
filter_daily_values = df[(df['Date'] > '2022-06-24') & (df['Date'] <= unique_date[-2])][['Date', 'Hour', 'Power (KW)']]
filter_daily_values

,Date,Hour,Power (KW)
1,2022-06-25,0,0.0
2,2022-06-25,1,0.0
3,2022-06-25,1,0.0
4,2022-06-25,1,0.0
5,2022-06-25,1,0.0
...,...,...,...
6591,2022-06-29,22,0.0
6592,2022-06-29,22,0.0
6593,2022-06-29,22,0.0
6594,2022-06-29,23,0.0


In [8]:
daily_hourly_values = filter_daily_values.groupby(['Date', 'Hour'])['Power (KW)'].sum().reset_index()
daily_hourly_values

,Date,Hour,Power (KW)
0,2022-06-25,0,0.000000
1,2022-06-25,1,0.000000
2,2022-06-25,2,0.000000
3,2022-06-25,3,0.000000
4,2022-06-25,4,0.000000
...,...,...,...
115,2022-06-29,19,0.144526
116,2022-06-29,20,0.036204
117,2022-06-29,21,0.000000
118,2022-06-29,22,0.000000


In [9]:
# daily_values = daily_values[daily_values['Date'] > '2022-06-11']

In [10]:
header_list = ['Date', 'Time', 'SolarIrradiance (W/m2)', 'weather status', 'Temp (°C)', 'RealFeelTemp (°C)', 'DewPoint (°C)', 'Wind (km/h)',
              'Direction', 'Hum (%)', 'Visibility (km)', 'UVIndex', 'UVIndexText', 'PreProbability (%)', 'RainProbability (%)',
              'CloudCover (%)']
weather_data = pd.read_csv('hourly_weather_forecasted_data.csv', names = header_list, encoding= 'unicode_escape')
weather_data.drop(['Date', 'Time', 'RealFeelTemp (°C)', 'DewPoint (°C)', 'Wind (km/h)', 'Direction', 'Visibility (km)', 'UVIndex', 
                   'UVIndexText', 'PreProbability (%)', 'RainProbability (%)', 'weather status'], axis = 1, inplace = True)
weather_data.tail(14)

,SolarIrradiance (W/m2),Temp (°C),Hum (%),CloudCover (%)
130,279.1,15.4,74,70
131,302.8,16.5,65,70
132,283.4,17.4,61,78
133,240.7,19.0,56,89
134,229.1,18.0,67,91
135,211.4,18.2,65,93
136,188.2,17.6,71,95
137,200.4,17.2,68,85
138,195.4,16.8,65,75
139,169.6,15.9,68,65


In [11]:
df1 = pd.concat([daily_hourly_values, weather_data], axis = 1)
df1.drop(['Date', 'Hour'], axis = 1, inplace = True)
df1.loc[df1['SolarIrradiance (W/m2)'] == 0, ['Temp (°C)', 'Hum (%)', 'CloudCover (%)']] = 0
# df1.to_csv('solar_energy_predictions.csv')
df1.tail(25)

,Power (KW),SolarIrradiance (W/m2),Temp (°C),Hum (%),CloudCover (%)
119,0.0,0.0,0.0,0,0
120,NaN,0.0,0.0,0,0
121,NaN,0.0,0.0,0,0
122,NaN,0.0,0.0,0,0
123,NaN,0.0,0.0,0,0
124,NaN,0.0,0.0,0,0
125,NaN,20.0,10.6,94,75
126,NaN,115.9,11.3,95,50
127,NaN,243.7,12.3,92,25
128,NaN,203.8,13.5,85,70


In [12]:
count_total_rows = len(df1) - 24

In [13]:
independent_columns = df1[['SolarIrradiance (W/m2)', 'Temp (°C)', 'Hum (%)', 'CloudCover (%)']][0:count_total_rows]

In [14]:
dependent_column = df1['Power (KW)'][0:count_total_rows]

In [15]:
rfr = RandomForestRegressor(n_estimators = 100, random_state = 0)
rfr.fit(independent_columns, dependent_column)

RandomForestRegressor(random_state=0)

#### Is model well fit or not?

In [16]:
r_sq = rfr.score(independent_columns, dependent_column)
r_sq

0.9695176586539458

In [17]:
forcasted_data = df1[['SolarIrradiance (W/m2)', 'Temp (°C)', 'Hum (%)', 'CloudCover (%)']].tail(24)

In [18]:
return_array = list(rfr.predict(forcasted_data))

In [19]:
# predicted_data = pd.DataFrame(return_array, columns = ['Power (KW)'])
# predicted_data

In [20]:
now = datetime.now()
date = now.strftime('%Y-%m-%d')
current_date_24 = [date, date, date, date, date, date, date, date, date, date, date, date, date, date, date, date,
                date, date, date, date, date, date, date, date]
current_date_12 = [date, date, date, date, date, date, date, date, date, date, date, date]

In [21]:
hours_24 = list(daily_hourly_values['Hour'][0:24])
hours_12 = list(daily_hourly_values['Hour'][0:12])

In [22]:
data_dict = {'Date': current_date_24, 'Hour': hours_24, 'Power (KW)': return_array}

In [23]:
data_dataframe = pd.DataFrame(data_dict)
data_dataframe

,Date,Hour,Power (KW)
0,2022-06-30,0,0.000000
1,2022-06-30,1,0.000000
2,2022-06-30,2,0.000000
3,2022-06-30,3,0.000000
4,2022-06-30,4,0.000000
5,2022-06-30,5,0.009950
6,2022-06-30,6,0.532499
7,2022-06-30,7,1.532226
8,2022-06-30,8,1.849988
9,2022-06-30,9,1.902435


In [24]:
data_dataframe['Power (KW)'].sum()

25.504786009392724

## Evaluating the model

#### MSE = mean_squared_error(actual value, predicted value)

In [25]:
mean_sq_error = metrics.mean_squared_error(today_hourly_values['Power (KW)'], data_dataframe['Power (KW)'].head(length_today_hourly_values))
mean_sq_error

0.6022046060570224

In [26]:
root_mean_sq_error = np.sqrt(mean_sq_error)
root_mean_sq_error

0.7760184315188798

In [27]:
mean_ab_error = metrics.mean_absolute_error(today_hourly_values['Power (KW)'], data_dataframe['Power (KW)'].head(length_today_hourly_values))
mean_ab_error

0.5266923643512661

#### R squared

In [28]:
r_squared = metrics.r2_score(today_hourly_values['Power (KW)'], data_dataframe['Power (KW)'].head(length_today_hourly_values))
r_squared

0.28771131118709525

#### Yesterday results

In [29]:
filter_last_day_values = df[df['Date'] == unique_date[-2]][['Date', 'Hour', 'Power (KW)']]
last_day_hourly_values = filter_last_day_values.groupby(['Date', 'Hour'])['Power (KW)'].sum().reset_index()

filter_yes_values = df[(df['Date'] >= '2022-06-25') & (df['Date'] <= unique_date[-3])][['Date', 'Hour', 'Power (KW)']]
yes_hourly_values = filter_yes_values.groupby(['Date', 'Hour'])['Power (KW)'].sum().reset_index()
header_list = ['Date', 'Time', 'SolarIrradiance (W/m2)', 'weather status', 'Temp (°C)', 'RealFeelTemp (°C)', 'DewPoint (°C)', 'Wind (km/h)',
              'Direction', 'Hum (%)', 'Visibility (km)', 'UVIndex', 'UVIndexText', 'PreProbability (%)', 'RainProbability (%)',
              'CloudCover (%)']
weather_data1 = pd.read_csv('hourly_weather_forecasted_data.csv', names = header_list, encoding= 'unicode_escape')
weather_unique_date = weather_data1['Date'].unique()
filter_weather_yes_values = weather_data1[
                                         (weather_data1['Date'] >= '2022-06-25') & 
                                         (weather_data1['Date'] <= weather_unique_date[-3])][['SolarIrradiance (W/m2)',
                                                                                             'Temp (°C)', 'Hum (%)',
                                                                                             'CloudCover (%)']]
yes_df1 = pd.concat([yes_hourly_values, filter_weather_yes_values], axis = 1)
yes_df1.drop(['Date', 'Hour'], axis = 1, inplace = True)
yes_df1.loc[yes_df1['SolarIrradiance (W/m2)'] == 0, ['Temp (°C)', 'Hum (%)', 'CloudCover (%)']] = 0
yes_count_total_rows = len(yes_df1)
yes_independent_columns = yes_df1[['SolarIrradiance (W/m2)', 'Temp (°C)', 'Hum (%)', 'CloudCover (%)']][0:yes_count_total_rows]
yes_dependent_column = yes_df1['Power (KW)'][0:yes_count_total_rows]
rfr_yes = RandomForestRegressor(n_estimators = 100, random_state = 0)
rfr_yes.fit(yes_independent_columns, yes_dependent_column)
forcasted_yes_values = weather_data1[(weather_data1['Date'] == weather_unique_date[-2])][['SolarIrradiance (W/m2)',
                                                                                        'Temp (°C)', 'Hum (%)',
                                                                                        'CloudCover (%)']]
forcasted_yes_values.loc[forcasted_yes_values['SolarIrradiance (W/m2)'] == 0, ['Temp (°C)', 'Hum (%)', 'CloudCover (%)']] = 0
rfr_yes_return_array = rfr_yes.predict(forcasted_yes_values)
rfr_yes_predicted_data = pd.DataFrame(rfr_yes_return_array, columns = ['Power (KW)'])
rfr_yes_pe = rfr_yes_predicted_data['Power (KW)'].sum()
rfr_yes_mse = metrics.mean_squared_error(last_day_hourly_values['Power (KW)'], rfr_yes_predicted_data['Power (KW)'])
rfr_yes_rmse = np.sqrt(rfr_yes_mse)
rfr_yes_mae = metrics.mean_absolute_error(last_day_hourly_values['Power (KW)'], rfr_yes_predicted_data['Power (KW)'])
rfr_yes_rs = metrics.r2_score(last_day_hourly_values['Power (KW)'], rfr_yes_predicted_data['Power (KW)'])
rfr_yes_mse

0.10700332381625484